# Beta Estimation Using Comparable Firms

**Target Firm: Lululemon (LULU)**

This notebook compares two methods of estimating beta:
1. **Bottom-up beta** using comparable firms
2. **Traditional regression beta** using historical returns

---

## Section 1: Imports and Setup

Install and import all necessary packages for the analysis.

In [ ]:
# Install required packages (uncomment if needed)
# !pip install yfinance pandas numpy matplotlib seaborn scipy statsmodels openai ipywidgets

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import yfinance as yf
import openai
import requests
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import gaussian_kde
import statsmodels.api as sm
from ipywidgets import interact, IntSlider
import warnings
warnings.filterwarnings('ignore')

# Set display options for better readability
pd.options.display.float_format = '{:,.4f}'.format
plt.style.use('seaborn-v0_8-darkgrid')

print("✓ All packages imported successfully")

## Section 2: Configuration and Parameters

**Set target firm and key parameters here.** This makes the notebook reusable for other firms.

In [ ]:
# ============================================================================
# TARGET FIRM CONFIGURATION
# ============================================================================

# Target firm ticker symbol
TARGET_TICKER = 'LULU'

# Marginal tax rate (U.S. corporate tax rate)
MARGINAL_TAX_RATE = 0.21

# OpenAI API Key (set your API key here if using OpenAI)
# openai.api_key = 'your-api-key-here'  # Uncomment and add your key

# ============================================================================

print("=" * 80)
print(f"BETA ESTIMATION ANALYSIS FOR {TARGET_TICKER}")
print("=" * 80)
print(f"\nParameters:")
print(f"  - Target Firm: {TARGET_TICKER}")
print(f"  - Marginal Tax Rate: {MARGINAL_TAX_RATE:.1%}")
print("=" * 80)

---
# Part 1: Use OpenAI API to Get Comparables

We'll use OpenAI's API to identify comparable firms for LULU.

In [ ]:
def get_comparable_firms_with_openai(target_ticker, target_description, num_firms=25):
    """
    Use OpenAI API to generate a list of comparable firms.
    
    Parameters:
    -----------
    target_ticker : str
        The ticker symbol of the target firm
    target_description : str
        Description of the target firm's business
    num_firms : int
        Number of comparable firms to identify
    
    Returns:
    --------
    str : Python code that creates a dataframe with comparable firms
    """
    
    prompt = f"""
    I need to find {num_firms} publicly traded companies that are comparable to {target_ticker} 
    ({target_description}) for the purpose of estimating beta in a valuation analysis.
    
    Please provide Python code that creates a pandas DataFrame with the following columns:
    - Firm Name: The company name
    - Description: Brief description of what the company does
    - Ticker Symbol: The stock ticker symbol
    
    The code should:
    1. Import pandas as pd
    2. Create a dictionary with the data
    3. Convert it to a DataFrame named 'df_comparable_firms'
    4. The final line should just be 'df_comparable_firms' to display it
    
    Focus on companies that:
    - Operate in similar industries (athletic apparel, sportswear, lifestyle brands)
    - Have similar business models
    - Are publicly traded with available stock data
    - Include a mix of pure-play competitors and broader apparel companies
    
    Provide ONLY the Python code, no explanations.
    """
    
    try:
        # Note: This requires a valid OpenAI API key
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a financial analyst assistant that provides clean Python code."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            max_tokens=2000
        )
        
        code = response.choices[0].message.content
        # Remove markdown code blocks if present
        if "```python" in code:
            code = code.split("```python")[1].split("```")[0]
        elif "```" in code:
            code = code.split("```")[1].split("```")[0]
        
        return code.strip()
    
    except Exception as e:
        print(f"Error calling OpenAI API: {e}")
        print("Please ensure you have set your OpenAI API key correctly.")
        return None

In [ ]:
# Uncomment the following lines if you want to use OpenAI API
# LULU is an athletic apparel company focused on yoga and athletic wear

"""
lulu_description = '''Lululemon Athletica Inc. is a premium athletic apparel company 
specializing in yoga-inspired athletic wear, technical athletic apparel, and lifestyle 
products with a focus on quality, innovation, and community engagement.'''

openai_code = get_comparable_firms_with_openai(TARGET_TICKER, lulu_description, 25)

if openai_code:
    print("Generated code from OpenAI:")
    print("-" * 80)
    print(openai_code)
    print("-" * 80)
    print("\n** Copy the code above and paste it in the next cell to create the dataframe **")
"""

### Discussion Question 1

**Describe what you are doing with your prompt above. Why do you expect it to give the results you want?**

**Answer:**

The prompt above is designed to give us relevant comparable firms by:

1. **SPECIFICITY**: We provide specific details about LULU's business model (athletic apparel, yoga-inspired, premium positioning) so OpenAI understands what makes a good comparable.

2. **STRUCTURED OUTPUT**: We request Python code that creates a DataFrame with specific columns (Firm Name, Description, Ticker Symbol), making the output immediately usable in our analysis.

3. **SELECTION CRITERIA**: We explicitly ask for companies that:
   - Operate in similar industries (athletic apparel, sportswear)
   - Have similar business models (retail, direct-to-consumer, premium positioning)
   - Are publicly traded (so we can get their financial data)
   - Include both pure-play competitors and broader apparel companies for diversity

4. **CODE FORMAT**: We request only Python code with no explanations, making it easy to copy-paste directly into our notebook.

This approach leverages OpenAI's knowledge of public companies and industry classifications while ensuring the output is in a format we can immediately use for our quantitative analysis.

---
# Part 2: Process Comparables List

Regardless of OpenAI output, we'll use a standardized list to ensure consistency.

In [ ]:
# Define the data (provided list to ensure consistency)
data = {
    'Ticker Symbol': [
        'NKE', 'UAA', 'ADS.DE', 'PVH', 'VFC',
        'COLM', 'PUM.DE', 'TPR', 'HBI', 'LEVI',
        'RL', 'GPS', 'URBN', 'ANF', 'GES',
        'LB', 'AEO', 'LULU', 'TJX', 'ROST',
        'COH', 'CPRI', 'FOSL', 'SKX', 'GIL'
    ]
}

# Create DataFrame
df_comparable_firms = pd.DataFrame(data)

print("Initial list of comparable firms:")
print(f"Total firms: {len(df_comparable_firms)}")
df_comparable_firms

In [ ]:
# Remove firms that are not suitable comparables
# - COLM (Columbia Sportswear): Different market positioning (outdoor vs. athletic)
# - TJX (TJ Maxx): Off-price retailer with different business model  
# - ROST (Ross Stores): Off-price retailer with different business model
# - LULU: This is our target firm, so we must exclude it from comparables

firms_to_remove = ['COLM', 'TJX', 'ROST', 'LULU']

print(f"Removing the following firms:")
for firm in firms_to_remove:
    print(f"  - {firm}")

# Drop the firms from the dataframe
df_comparable_firms = df_comparable_firms[~df_comparable_firms['Ticker Symbol'].isin(firms_to_remove)]

# Reset index for clean dataframe
df_comparable_firms = df_comparable_firms.reset_index(drop=True)

print(f"\n\nFinal list of comparable firms:")
print(f"Total firms: {len(df_comparable_firms)}")
df_comparable_firms

---
# Part 3: Get Financial Data for Target Firm

Get Beta, Market Cap, and Total Debt for LULU using yfinance.

In [ ]:
def get_firm_data(ticker):
    """
    Get Beta, Market Cap, and Total Debt for a given ticker using yfinance.
    
    Parameters:
    -----------
    ticker : str
        Stock ticker symbol
    
    Returns:
    --------
    dict : Dictionary with 'beta', 'market_cap', and 'total_debt'
    """
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        
        # Get beta (5-year monthly beta)
        beta = info.get('beta', None)
        
        # Get market capitalization
        market_cap = info.get('marketCap', None)
        
        # Get total debt from balance sheet
        total_debt = info.get('totalDebt', None)
        
        if total_debt is None:
            # Try to get from balance sheet if not in info
            try:
                bs = stock.balance_sheet
                if not bs.empty and 'Total Debt' in bs.index:
                    total_debt = bs.loc['Total Debt'].iloc[0]
                elif not bs.empty and 'Long Term Debt' in bs.index:
                    total_debt = bs.loc['Long Term Debt'].iloc[0]
            except:
                total_debt = None
        
        return {
            'ticker': ticker,
            'beta': beta,
            'market_cap': market_cap,
            'total_debt': total_debt
        }
    
    except Exception as e:
        print(f"Error getting data for {ticker}: {e}")
        return None

In [ ]:
# Get data for target firm
print(f"Fetching financial data for {TARGET_TICKER}...\n")
target_data = get_firm_data(TARGET_TICKER)

if target_data:
    target_beta = target_data['beta']
    target_market_cap = target_data['market_cap']
    target_total_debt = target_data['total_debt']
    
    print(f"{TARGET_TICKER} Financial Data:")
    print(f"  Beta (5Y Monthly): {target_beta:.4f}" if target_beta else "  Beta: N/A")
    print(f"  Market Cap: ${target_market_cap:,.0f}" if target_market_cap else "  Market Cap: N/A")
    print(f"  Total Debt: ${target_total_debt:,.0f}" if target_total_debt else "  Total Debt: N/A")
    
    # Calculate D/E ratio for target
    if target_market_cap and target_total_debt:
        target_de_ratio = target_total_debt / target_market_cap
        print(f"  D/E Ratio: {target_de_ratio:.4f}")
else:
    print(f"Failed to retrieve data for {TARGET_TICKER}")

---
# Part 4: Get Financial Data for All Comparable Firms

Loop through each comparable firm and collect Beta, Market Cap, and Total Debt.

### Discussion Question 2

**How did you "collect" what you needed in each iteration of the for loop so that when it finished, you could print the final product? Describe it in detail.**

**Answer:**

To 'collect' data from each iteration of the for loop, I use the following approach:

1. **CREATE EMPTY LIST**: Before the loop starts, I initialize an empty list called `comparable_data` that will store dictionaries of financial data.

2. **ITERATE AND COLLECT**: For each ticker in the dataframe:
   - Call the `get_firm_data()` function to retrieve Beta, Market Cap, and Total Debt
   - If successful and all required data is available, append the results dictionary to the `comparable_data` list
   - If there's an error or missing data, skip that ticker and continue

3. **ERROR HANDLING**: Using try-except blocks ensures that if a ticker is invalid, delisted, or missing data, the loop continues rather than crashing. This is crucial because OpenAI might suggest tickers that are no longer valid or have incomplete data.

4. **CONVERT TO DATAFRAME**: After the loop completes, I convert the list of dictionaries into a pandas DataFrame using `pd.DataFrame(comparable_data)`. This creates a clean dataframe with only valid data.

This approach ensures we end up with a usable dataset containing only firms with complete financial information, while gracefully handling any data quality issues.

In [ ]:
# Initialize list to store data for all comparable firms
comparable_data = []

print("Fetching data for comparable firms...")
print("-" * 80)

# Iterate through each comparable firm
for idx, row in df_comparable_firms.iterrows():
    ticker = row['Ticker Symbol']
    print(f"Processing {ticker}...", end=" ")
    
    # Get financial data for this ticker
    firm_data = get_firm_data(ticker)
    
    # Only add to our list if we got valid data
    if firm_data and all([
        firm_data['beta'] is not None,
        firm_data['market_cap'] is not None,
        firm_data['total_debt'] is not None
    ]):
        comparable_data.append(firm_data)
        print(f"✓ Success (Beta: {firm_data['beta']:.4f})")
    else:
        print(f"✗ Skipped (missing data)")

print("-" * 80)

# Create dataframe from collected data
df_comps = pd.DataFrame(comparable_data)

# Rename columns for clarity
df_comps.columns = ['Ticker', 'Beta', 'Market Cap', 'Total Debt']

print(f"\n\nSuccessfully retrieved data for {len(df_comps)} firms:")
df_comps

---
# Part 5: Compute D/E Ratio and Unlever Beta

Calculate the Debt-to-Equity ratio and unlever the betas for each comparable firm.

In [ ]:
# Calculate D/E Ratio: D/E = Total Debt / Market Cap
df_comps['D/E Ratio'] = df_comps['Total Debt'] / df_comps['Market Cap']

# Calculate Unlevered Beta
# Formula: Unlevered Beta = Beta / (1 + D/E Ratio * (1 - tax_rate))
df_comps['Unlevered Beta'] = df_comps['Beta'] / (1 + df_comps['D/E Ratio'] * (1 - MARGINAL_TAX_RATE))

print("Comparable Firms with D/E Ratios and Unlevered Betas:")
print("=" * 80)
df_comps

In [ ]:
# Summary statistics
print("Summary Statistics:")
print(f"  Average Unlevered Beta: {df_comps['Unlevered Beta'].mean():.4f}")
print(f"  Median Unlevered Beta: {df_comps['Unlevered Beta'].median():.4f}")
print(f"  Std Dev Unlevered Beta: {df_comps['Unlevered Beta'].std():.4f}")
print(f"  Min Unlevered Beta: {df_comps['Unlevered Beta'].min():.4f}")
print(f"  Max Unlevered Beta: {df_comps['Unlevered Beta'].max():.4f}")

---
# Part 6: Compute Levered Beta for Target Firm

Relever each comparable's unlevered beta using the target firm's D/E ratio.

In [ ]:
# Compute target firm's D/E Ratio
target_de_ratio = target_total_debt / target_market_cap
print(f"{TARGET_TICKER} D/E Ratio: {target_de_ratio:.4f}\n")

# Relever each comparable's unlevered beta using target firm's D/E ratio
# Formula: Levered Beta = Unlevered Beta * (1 + Target D/E Ratio * (1 - tax_rate))
df_comps['Levered Beta (Target D/E)'] = df_comps['Unlevered Beta'] * (
    1 + target_de_ratio * (1 - MARGINAL_TAX_RATE)
)

print("Comparable Firms with Relevered Betas:")
df_comps[['Ticker', 'Beta', 'Unlevered Beta', 'Levered Beta (Target D/E)']]

In [ ]:
# Calculate average levered beta (this is our bottom-up beta estimate)
bottom_up_beta = df_comps['Levered Beta (Target D/E)'].mean()

print("=" * 80)
print(f"BOTTOM-UP BETA ESTIMATE FOR {TARGET_TICKER}: {bottom_up_beta:.4f}")
print("=" * 80)

---
# Part 7: Get Risk-Free Rate

Get the current 10-year Treasury yield to use as the risk-free rate.

In [ ]:
def get_risk_free_rate():
    """
    Get the current 10-year Treasury yield as the risk-free rate.
    Uses yfinance to get ^TNX (10-year Treasury yield).
    
    Returns:
    --------
    float : Risk-free rate as a decimal
    """
    try:
        # Get 10-year Treasury yield (^TNX is the ticker for 10-year Treasury yield)
        treasury = yf.Ticker("^TNX")
        
        # Get the most recent close price (this is the yield in percentage)
        hist = treasury.history(period="5d")
        
        if not hist.empty:
            risk_free_rate = hist['Close'].iloc[-1] / 100  # Convert from percentage to decimal
            return risk_free_rate
        else:
            # If data not available, use a typical long-term average
            print("Unable to fetch current rate, using default 4.5%")
            return 0.045
    
    except Exception as e:
        print(f"Error fetching risk-free rate: {e}")
        print("Using default 4.5%")
        return 0.045

risk_free_rate = get_risk_free_rate()
print(f"Risk-Free Rate (10-Year Treasury): {risk_free_rate:.2%}")

---
# Part 8: Compute Regression Beta Using OLS

Get historical data and perform OLS regression to compute the traditional beta.

In [ ]:
# Calculate date range (last 5 years)
end_date = datetime.now()
start_date = end_date - timedelta(days=5*365)

print(f"Fetching monthly data from {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}\n")

# Get monthly data for target firm and market (S&P 500)
target_stock = yf.download(TARGET_TICKER, start=start_date, end=end_date, interval='1mo', progress=False)
market_stock = yf.download('^GSPC', start=start_date, end=end_date, interval='1mo', progress=False)

# Calculate monthly returns
target_returns = target_stock['Adj Close'].pct_change().dropna()
market_returns = market_stock['Adj Close'].pct_change().dropna()

# Align the data (ensure same dates)
returns_df = pd.DataFrame({
    'Target': target_returns,
    'Market': market_returns
}).dropna()

print(f"Number of monthly observations: {len(returns_df)}")

In [ ]:
# Calculate Equity Market Risk Premium (EMRP)
# EMRP = Market Return - Risk-Free Rate (monthly)
risk_free_rate_monthly = (1 + risk_free_rate) ** (1/12) - 1
returns_df['EMRP'] = returns_df['Market'] - risk_free_rate_monthly

# Calculate excess returns for target firm
returns_df['Excess_Return'] = returns_df['Target'] - risk_free_rate_monthly

# Display first few rows
print("Sample of returns data:\n")
returns_df.head(10)

In [ ]:
# Perform OLS Regression
# Y = Excess Returns of Target
# X = EMRP (Equity Market Risk Premium)
X = returns_df['EMRP']
Y = returns_df['Excess_Return']

# Add constant for intercept (alpha)
X_with_const = sm.add_constant(X)

# Fit the model
model = sm.OLS(Y, X_with_const)
results = model.fit()

print("=" * 80)
print("OLS REGRESSION RESULTS")
print("=" * 80)
print(results.summary())

In [ ]:
# Extract beta (coefficient on EMRP)
regression_beta = results.params['EMRP']
regression_beta_stderr = results.bse['EMRP']

print("=" * 80)
print(f"REGRESSION BETA FOR {TARGET_TICKER}: {regression_beta:.4f}")
print(f"Standard Error: {regression_beta_stderr:.4f}")
print("=" * 80)

---
# Part 9: Compare Beta Estimates and Uncertainty

Compare the bottom-up beta with the regression beta, including their uncertainties.

In [ ]:
# Calculate standard error for bottom-up beta
bottom_up_std = df_comps['Levered Beta (Target D/E)'].std()
bottom_up_n = len(df_comps)
bottom_up_stderr = bottom_up_std / np.sqrt(bottom_up_n)

print("=" * 80)
print("BETA COMPARISON SUMMARY")
print("=" * 80)
print(f"\nBottom-Up Beta (from Comparables):")
print(f"  Estimate: {bottom_up_beta:.4f}")
print(f"  Standard Error: {bottom_up_stderr:.4f}")
print(f"  Number of Comparables: {bottom_up_n}")
print(f"  95% Confidence Interval: [{bottom_up_beta - 1.96*bottom_up_stderr:.4f}, {bottom_up_beta + 1.96*bottom_up_stderr:.4f}]")

print(f"\nRegression Beta (5-Year Monthly):")
print(f"  Estimate: {regression_beta:.4f}")
print(f"  Standard Error: {regression_beta_stderr:.4f}")
print(f"  R-squared: {results.rsquared:.4f}")
print(f"  95% Confidence Interval: [{regression_beta - 1.96*regression_beta_stderr:.4f}, {regression_beta + 1.96*regression_beta_stderr:.4f}]")

print(f"\nDifference:")
print(f"  Absolute: {abs(bottom_up_beta - regression_beta):.4f}")
print(f"  Relative: {(bottom_up_beta - regression_beta)/regression_beta * 100:.2f}%")

In [ ]:
# Create KDE plot comparing both distributions
fig, ax = plt.subplots(figsize=(12, 6))

# Generate x values for smooth plotting
x_min = min(bottom_up_beta - 3*bottom_up_stderr, regression_beta - 3*regression_beta_stderr)
x_max = max(bottom_up_beta + 3*bottom_up_stderr, regression_beta + 3*regression_beta_stderr)
x_range = np.linspace(x_min, x_max, 1000)

# Bottom-Up Beta Distribution (normal distribution)
bottom_up_pdf = stats.norm.pdf(x_range, bottom_up_beta, bottom_up_stderr)
ax.plot(x_range, bottom_up_pdf, label=f'Bottom-Up Beta (n={bottom_up_n})',
        linewidth=2.5, color='#2E86AB', alpha=0.8)
ax.fill_between(x_range, bottom_up_pdf, alpha=0.3, color='#2E86AB')

# Regression Beta Distribution (normal distribution)
regression_pdf = stats.norm.pdf(x_range, regression_beta, regression_beta_stderr)
ax.plot(x_range, regression_pdf, label=f'Regression Beta (5-Year)',
        linewidth=2.5, color='#A23B72', alpha=0.8)
ax.fill_between(x_range, regression_pdf, alpha=0.3, color='#A23B72')

# Add vertical lines for means
ax.axvline(bottom_up_beta, color='#2E86AB', linestyle='--', linewidth=1.5, alpha=0.7)
ax.axvline(regression_beta, color='#A23B72', linestyle='--', linewidth=1.5, alpha=0.7)

# Formatting
ax.set_xlabel('Beta', fontsize=12, fontweight='bold')
ax.set_ylabel('Density', fontsize=12, fontweight='bold')
ax.set_title(f'Distribution of Beta Estimates for {TARGET_TICKER}',
             fontsize=14, fontweight='bold', pad=20)
ax.legend(fontsize=11, loc='upper right', framealpha=0.95)
ax.grid(True, alpha=0.3, linestyle='--')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.tight_layout()
plt.show()

---
# Interactive Plot: Varying Number of Comparables

Use the slider to see how the number of comparable firms affects the precision of the bottom-up beta estimate.

In [ ]:
def plot_beta_distributions(n):
    """
    Create KDE plot with adjustable number of comparable firms.
    
    Parameters:
    -----------
    n : int
        Number of comparable firms to include in bottom-up beta calculation
    """
    # Select first n firms from our comparable list
    df_subset = df_comps.head(n)
    
    # Recalculate bottom-up beta with subset
    subset_beta = df_subset['Levered Beta (Target D/E)'].mean()
    subset_std = df_subset['Levered Beta (Target D/E)'].std()
    subset_stderr = subset_std / np.sqrt(n)
    
    # Create figure
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Generate x values
    x_min = min(subset_beta - 3*subset_stderr, regression_beta - 3*regression_beta_stderr) - 0.1
    x_max = max(subset_beta + 3*subset_stderr, regression_beta + 3*regression_beta_stderr) + 0.1
    x_range = np.linspace(x_min, x_max, 1000)
    
    # Bottom-Up Beta Distribution
    bottom_up_pdf = stats.norm.pdf(x_range, subset_beta, subset_stderr)
    ax.plot(x_range, bottom_up_pdf, label=f'Bottom-Up Beta (n={n})',
            linewidth=2.5, color='#2E86AB', alpha=0.8)
    ax.fill_between(x_range, bottom_up_pdf, alpha=0.3, color='#2E86AB')
    
    # Regression Beta Distribution (stays constant)
    regression_pdf = stats.norm.pdf(x_range, regression_beta, regression_beta_stderr)
    ax.plot(x_range, regression_pdf, label=f'Regression Beta (5-Year)',
            linewidth=2.5, color='#A23B72', alpha=0.8)
    ax.fill_between(x_range, regression_pdf, alpha=0.3, color='#A23B72')
    
    # Add vertical lines for means
    ax.axvline(subset_beta, color='#2E86AB', linestyle='--', linewidth=1.5, alpha=0.7,
              label=f'Bottom-Up: {subset_beta:.4f} ± {subset_stderr:.4f}')
    ax.axvline(regression_beta, color='#A23B72', linestyle='--', linewidth=1.5, alpha=0.7,
              label=f'Regression: {regression_beta:.4f} ± {regression_beta_stderr:.4f}')
    
    # Formatting
    ax.set_xlabel('Beta', fontsize=12, fontweight='bold')
    ax.set_ylabel('Density', fontsize=12, fontweight='bold')
    ax.set_title(f'Distribution of Beta Estimates for {TARGET_TICKER} (n={n} comparables)',
                 fontsize=14, fontweight='bold', pad=20)
    ax.legend(fontsize=10, loc='upper right', framealpha=0.95)
    ax.grid(True, alpha=0.3, linestyle='--')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    plt.tight_layout()
    plt.show()
    
    # Print statistics
    print(f"\nWith n={n} comparables:")
    print(f"  Bottom-Up Beta: {subset_beta:.4f} (SE: {subset_stderr:.4f})")
    print(f"  Distribution Width (±2σ): {4*subset_stderr:.4f}")

# Create interactive widget
print("** Use the slider below to adjust the number of comparable firms **")
print("Notice how the bottom-up beta distribution becomes narrower as n increases.\n")

interact(plot_beta_distributions,
         n=IntSlider(min=3, max=len(df_comps), step=1, value=len(df_comps),
                    description='# of Comps:', continuous_update=False))

---
# Final Discussion Questions

### Discussion Question 3

**Using the interactive plot above, what n do you think is appropriate? I.e., how many beta comps do you need to get a precise enough estimate of beta?**

**Answer:**

Based on the interactive plot, an appropriate value of n (number of comparable firms) appears to be around **10-15 firms**. Here's the reasoning:

1. **PRECISION IMPROVEMENT**: As n increases from 3 to ~10, we see substantial reduction in standard error, making our estimate more precise. The distribution becomes notably narrower and more peaked.

2. **DIMINISHING RETURNS**: Beyond n ≈ 10-15, additional comparables provide diminishing marginal improvement in precision. The distribution still narrows, but much more slowly.

3. **QUALITY vs QUANTITY**: Having too many comparables (n > 15) may dilute the analysis by including less-similar firms. It's better to have 10 highly comparable firms than 20 firms where some are only marginally similar.

4. **PRACTICAL BALANCE**: With 10-15 comparables, we achieve:
   - Sufficient statistical power to estimate beta reliably
   - Narrow enough confidence intervals for practical decision-making
   - Diversity to capture industry characteristics
   - Avoidance of firm-specific idiosyncrasies

5. **COMPARISON TO REGRESSION**: At n ≈ 10-15, the bottom-up beta's precision becomes comparable to or better than the regression beta, while potentially being more stable (not influenced by temporary market conditions during the 5-year period).

### Discussion Question 4

**Not python related. Step back and think about all that we did here. What do you conclude from this plot (and entire notebook's analysis as a whole)?**

**Answer:**

This analysis reveals several important insights about beta estimation methods:

**1. METHODOLOGICAL COMPARISON:**
- The bottom-up (comparables) and regression approaches can yield different beta estimates
- Both methods are valid but capture different aspects of systematic risk
- The difference between methods highlights the inherent uncertainty in beta estimation

**2. UNCERTAINTY CONSIDERATIONS:**
- Both methods have measurable uncertainty (standard errors)
- The precision of bottom-up beta depends on the number of comparables used
- Understanding the distribution of possible beta values is as important as the point estimate

**3. ADVANTAGES OF BOTTOM-UP APPROACH:**
- **Forward-looking**: Based on current comparable firms' characteristics
- **More stable**: Not affected by temporary market conditions or thin trading
- **Useful for private companies**: Can estimate beta without historical price data
- **Adjustable**: Can easily update as leverage or business mix changes

**4. ADVANTAGES OF REGRESSION APPROACH:**
- **Firm-specific**: Captures the actual historical relationship with market
- **Well-established**: Standard methodology with clear statistical interpretation
- **R-squared** provides goodness-of-fit measure

**5. PRACTICAL IMPLICATIONS:**
- For valuation, considering both estimates or using an average may be prudent
- The uncertainty in beta estimation highlights why sensitivity analysis is crucial in DCF valuation
- Small differences in beta can meaningfully impact cost of equity and firm value

**6. KEY INSIGHT:**

**Beta estimation is as much art as science.** Both methods have merits, and the "true" beta is unknowable. Understanding the uncertainty around our estimate (shown by these distributions) is as important as the point estimate itself. This analysis demonstrates that we should always consider a range of possible betas in our valuation work rather than relying on a single point estimate. The convergence (or divergence) of the two methods provides valuable information about the reliability of our beta estimate.